In [ ]:
import torch
import torch.nn as nn
from networks import DynamicUNet, BaseUNet

network = DynamicUNet(1, 5, [5,4,3,2,1], [1,2,3,4,5])

network.use_higher_layer()
network.use_higher_layer()
network.use_higher_layer()
network.use_higher_layer()
network.use_higher_layer()

val = network(torch.randn((5, 1, 1024, 1024)))

print(val.size())


In [ ]:
from torchviz import make_dot

make_dot(val.sum(), params=dict(list(network.named_parameters()))).render("torchviz3", format="png")

In [ ]:

# temp_ = network.additional_mapper_classifiers[-2]["mapper"](torch.randn((5, 1, 128, 128)))

# add_down_ = network.additional_downscale[-2](temp_)
# downscaled_ = network.downscale(add_down_)

# #-----------------------------------#

# # temp = network.additional_mapper_classifiers[-1]["mapper"](torch.randn((5, 1, 64, 64)))

# add_down = network.additional_downscale[-1](downscaled_)
# downscaled = network.downscale(add_down)

# y_hat = network.base_unet(downscaled)

# upscaled = network.additional_upscale[-1]["upsample_conv"](y_hat)

# sth = torch.cat([upscaled, add_down], dim=1)

# res = network.additional_upscale[-1]["conv_cell"](sth)

# # res = network.additional_mapper_classifiers[-1]["classifier"](res)

# #-----------------------------------#

# upscaled = network.additional_upscale[-2]["upsample_conv"](res)

# sth = torch.cat([upscaled, add_down_], dim=1)

# res = network.additional_upscale[-2]["conv_cell"](sth)

# res = network.additional_mapper_classifiers[-2]["classifier"](res)
